In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import dcf
from yieldcurves.parametric import NelsonSiegelSvensson, \
    NelsonSiegelSvenssonShortRate
from yieldcurves.tools import ascii_plot, lin, plot
from yieldcurves import *
from yieldcurves.analytics import *
from yieldcurves.interpolation import linear, plist, \
    _default_value_interpolation, no
from yieldcurves.compounding import *

In [57]:
curve = linear([0.0, 0.2518822724161533, 0.5037645448323066, 0.75564681724846,
                0.999315537303217, 1.2511978097193703, 1.5030800821355237,
                1.754962354551677, 1.998631074606434, 2.2505133470225873,
                2.5023956194387407, 2.754277891854894, 2.9979466119096507,
                3.249828884325804, 3.5017111567419574, 3.753593429158111, 4.0,
                4.251882272416153, 4.503764544832307, 4.75564681724846,
                4.999315537303217, 5.25119780971937, 5.503080082135524,
                5.754962354551677, 5.9986310746064335, 6.250513347022587,
                6.50239561943874, 6.754277891854894, 6.997946611909651,
                7.2498288843258045, 7.501711156741957, 7.753593429158111, 8.0,
                8.251882272416154, 8.503764544832306, 8.75564681724846,
                8.999315537303216, 9.25119780971937, 9.503080082135524,
                99.],
               [0.02, 0.018, 0.016, 0.015, 0.017, 0.019, 0.02, 0.018, 0.016,
                0.015, 0.017, 0.019, 0.02, 0.018, 0.016, 0.015, 0.017, 0.019,
                0.02, 0.018, 0.016, 0.015, 0.017, 0.019, 0.02, 0.018, 0.016,
                0.015, 0.017, 0.019, 0.02, 0.018, 0.016, 0.015, 0.017, 0.019,
                0.02, 0.018, 0.016, 0.015])

dcf_curve = dcf.ZeroRateCurve(curve.x_list, curve.y_list, forward_tenor=0.25, origin=0)
yc_curve = ZeroRate(curve, cash_frequency=None, frequency=None)

self, start, stop = dcf_curve, 0.0, 0.0027378507871321013
print(f"{start=} {stop=}")
ir = self._get_compounding_rate(start, stop)
t = self.day_count(start, stop)
df = self._get_compounding_factor(start, stop)
print(f"{ir=} {t=} {df=}")
print(f"{self._get_compounding_rate(0, stop)=} {self._get_compounding_rate(0, start)=}")
print(f"{self._get_compounding_factor(0, stop)=} {self._get_compounding_factor(0, start)=}")

ir = yc_curve.zero(start, stop)
t = stop - start
df = yc_curve.df(start, stop)
print(f"{ir=} {t=} {df=}")

print(f"{yc_curve.curve(stop)=} {yc_curve.curve(start)=}")
print(f"{yc_curve.df(stop)=} {yc_curve.df(start)=}")

dcf_curve.get_short_rate(0.0027378507871321013), yc_curve.short(0.0027378507871321013)


start=0.0 stop=0.0027378507871321013
ir=0.019978260869565216 t=0.0027378507871321013 df=0.9999453039986336
self._get_compounding_rate(0, stop)=0.019978260869565216 self._get_compounding_rate(0, start)=0.02
self._get_compounding_factor(0, stop)=0.9999453039986336 self._get_compounding_factor(0, start)=1.0
ir=0.01997826086956044 t=0.0027378507871321013 df=0.9999453039986336
yc_curve.curve(stop)=0.019978260869565216 yc_curve.curve(start)=0.02
yc_curve.df(stop)=0.9999453039986336 yc_curve.df(start)=1.0


(0.018, 0.019934782608676094)

In [31]:
d = dcf.ZeroRateCurve(l.x_list, l.y_list, forward_tenor=0.25)
start, stop = x, y
d.get_discount_factor(x, y), d.get_cash_rate(x, y), d._get_compounding_factor(
    start, stop)

(0.994975022206863, 0.020050461154033597, 0.994975022206863)

In [ ]:
l = linear([1, 2, 3, 50], [0.01, 0.05, 0.02, 0.03])
print(repr(l))
str(l), repr(l), l(2.5), l.x_list

In [ ]:
# c = NelsonSiegelSvensson.download(-1)
c = NelsonSiegelSvensson(beta0=1.9137, beta1=1.97448, beta2=-28.16468,
                         beta3=27.22348, tau1=4.6538, tau2=5.29223)
c

In [ ]:
NelsonSiegelSvenssonShortRate(**c.params)

In [ ]:
a = AssetPriceYield(.03, spot=40)
a, a.price_yield(1, 2), a.price(10)

In [ ]:
z = ZeroRate(l)
df = Df(z)
z.par(5, 10)

In [ ]:
c = Const(0.01)
z = ZeroRate(c, cash_frequency=2)
df = Df(z)

In [ ]:
T = 3
dates = list(x / z.cash_frequency for x in range(T * z.cash_frequency)) + [T]
sum(df(x) / z.cash_frequency for x in dates[1:])

In [ ]:
swap_annuity(z, dates)

In [ ]:
swap_annuity(z, T, cash_frequency=z.cash_frequency)

In [ ]:
annuity = Annuity(z)
annuity, annuity(T)

In [ ]:
swap_par_rate(z, T, cash_frequency=z.cash_frequency)

In [ ]:
par = Par(z)
par, par(T), par(T, 15), z.par(T), z.par(T, 15), z.par

In [ ]:
Cash(z), Cash(z)(T), z.cash(T), z.cash

In [ ]:
Df(z).cash(5), z.cash(5)

In [ ]:
#_ = plot(lin(0.25, 30, 0.25), z, Par(z), Cash(z), Cash(z, cash_frequency=1),legend=True)
_ = plot(lin(0.25, 25, 0.25), Cash(z), Cash(z, cash_frequency=1), Par(z),
         Par(z, cash_frequency=1), legend=True)

In [ ]:
zcurve = ZeroRate(0.01)
df = Df(zcurve)
zcurve, df, df(5, 10), zcurve.df(5, 10), df.zero(5, 10), zcurve(5, 10)

In [ ]:
zcurve = ZeroRate(0.01)
fwd = CashRate(ZeroRate(0.01), frequency=4)
swp = Par(zcurve, forward_curve=fwd)
swp

In [ ]:
df = Df(zcurve)
zcurve.cash(10), df.cash(10), Cash(df)(10)

In [ ]:
z = ZeroRate(c, frequency=1)

In [ ]:
c = NelsonSiegelSvensson.download(-1)
setattr(c, 'repr_repr', r'$i(t)$')
d = NelsonSiegelSvensson.download(-1)
d.short_rate = True
setattr(d, 'repr_repr', r'$r(t)$')

In [ ]:
ascii_plot(lin(0, 30, 0.025), c, d)

In [ ]:
_ = plot(lin(0, 30, 0.025), c, d, legend=True)

In [ ]:
f = NelsonSiegelSvensson.download('2024-04-16')
f_1m = Cash(ZeroRate(f, frequency=12))
f_3m = Cash(ZeroRate(f, frequency=4))
f_6m = Cash(ZeroRate(f, frequency=2))
f12m = Cash(ZeroRate(f, frequency=1))
s = NelsonSiegelSvensson.download('2024-04-16')
s.short_rate = True
setattr(f, 'repr_repr', r'$i$')
setattr(f_1m, 'repr_repr', r'$f_{\tt 1m}$')
setattr(f_3m, 'repr_repr', r'$f_{\tt 3m}$')
setattr(f_6m, 'repr_repr', r'$f_{\tt 6m}$')
setattr(f12m, 'repr_repr', r'$f_{\tt 12m}$')
setattr(s, 'repr_repr', r'$r$')

In [ ]:
ax = plot(lin(0, 10, 0.025), f, f_1m, f_3m, f_6m, f12m, s, legend=True,
          show=False)
ax.set_ylim([0.02, 0.03])